In [1]:
import pandas as pd
import numpy as np
import nltk
from bs4 import BeautifulSoup

import re
import nltk
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer,TfidfVectorizer
from nltk import word_tokenize
import string 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import nmf
from nltk.corpus import stopwords
from nltk.tokenize import MWETokenizer,word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from nltk.corpus import wordnet
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split as tts,GridSearchCV
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from gensim.models import word2vec


ModuleNotFoundError: No module named 'nltk'

In [176]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
url = "https://raw.githubusercontent.com/reubence/IPYNB-Notebooks/master/datasets/Articles.csv"
df_art = pd.read_csv(url, encoding = "utf-8’")

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('german'))
nltk.download('words')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

In [181]:
df_art.head()

,ID_Article,Path,publishingDate,Title,Body
0,1,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro..."
1,2,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ..."
2,3,Diverses/mobil,2013-11-22 12:15:00.00,Die Android App von derStandard.at,"<div class=""section"" id=""content-main"" itempro..."
3,4,Newsroom/User/mitmachen/Mitreden,2014-08-13 05:30:00.00,Welche Erfahrungen haben Sie als Linkshänder g...,"<div class=""section"" id=""content-main"" itempro..."
4,5,Newsroom/User/mitmachen/Mitreden,2014-08-27 12:27:01.09,Wie haben Sie das Jahr 1989 erlebt?,"<div class=""section"" id=""content-main"" itempro..."


In [0]:
def to_lower(str): # steps... 
                 # 1.  Lowering and removing digits and separating gmail words because we dont want to read nicknames and emails
  text= str.lower() #1
  text = text.translate(text.maketrans('', '', string.digits))
  text = text.translate(text.maketrans('', '', '''!()-[]{};:'"\,<>./?@#$%^&*_~'''))
  str_new = " ".join(re.findall("[a-zA-Z]+", text))
  return str_new

In [0]:
 def tokenize(str_new):
      # 2. Tokenizing again because the result returns a string.. just to be safe
  tokens = [] #2
  tokens = nltk.word_tokenize(str_new)
  return tokens

In [0]:
def remove_stop(tokens):
  # 3. Removing stopwords using german corpus... also removinbg punctuations
  removed = [] #3
  stopword = stopwords.words('german') 
  punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  for w in tokens:
    if (w.lower() not in stopword and w.lower() not in punctuations):
      removed.append(w.lower())
  return removed

In [0]:
def pos_tag(removed):
           # 4. Pos Tagging for Lemmatizing
  pos = [] #4
  pos.append(nltk.pos_tag(removed))
  return pos

In [0]:
def lemma(pos):
# 5. Lemmatizing
               
  lem = WordNetLemmatizer() #5
  lemma = []
  for word in pos:
    for w in word:
      pos_value = ""
      if (w[1].startswith('J')):
        pos_value = wordnet.ADJ
      elif (w[1].startswith('V')):
        pos_value = wordnet.VERB
      elif (w[1].startswith('N')):
        pos_value = wordnet.NOUN
      elif (w[1].startswith('R')):
        pos_value = wordnet.ADV 
      else:
        continue
      lemma.append(lem.lemmatize(w[0],pos_value))
  return lemma
 

In [0]:
def to_string(lemma): # 6. Return Sentence instead of tokens
  str1 = ' '.join([elem for elem in lemma]) #6
  return (str1)

# Sklearn Pipeline using Function Transformer

In [0]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

pipeline = make_pipeline(
        FunctionTransformer(to_lower),
        FunctionTransformer(tokenize),
        FunctionTransformer(remove_stop),
        FunctionTransformer(pos_tag),
        FunctionTransformer(lemma),
        FunctionTransformer(to_string),
    )


In [190]:
#Selecting first 1000 rows only
df_art = df_art.head(1000)
df_art.head()

,ID_Article,Path,publishingDate,Title,Body
0,1,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro..."
1,2,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ..."
2,3,Diverses/mobil,2013-11-22 12:15:00.00,Die Android App von derStandard.at,"<div class=""section"" id=""content-main"" itempro..."
3,4,Newsroom/User/mitmachen/Mitreden,2014-08-13 05:30:00.00,Welche Erfahrungen haben Sie als Linkshänder g...,"<div class=""section"" id=""content-main"" itempro..."
4,5,Newsroom/User/mitmachen/Mitreden,2014-08-27 12:27:01.09,Wie haben Sie das Jahr 1989 erlebt?,"<div class=""section"" id=""content-main"" itempro..."


In [195]:
df_art["Title-Cleaned"] = df_art["Title"].apply(pipeline.fit_transform)
df_art["Title-Cleaned"]

0                               newsletter derstandardat
1                                     teil derstandardat
2                              android app derstandardat
3                        erfahrungen linksh nder gemacht
4                                            jahr erlebt
                             ...                        
995                 casinoschef konzern komplett umbauen
996                            window steht mobilversion
997    aufregung kettens genmassaker wieso neue doom ...
998        telekom austria pr ft pilzangaben nsaspionage
999    selbstfahrende googleautos ffentlichen stra un...
Name: Title-Cleaned, Length: 1000, dtype: object

In [0]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_art['Title-Cleaned'])
x = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())

In [0]:
nmf=NMF(4,random_state=42,alpha=0.1,l1_ratio=0.5,init='nndsvd')
arr=nmf.fit_transform(x)
h = nmf.components_


In [0]:
def display_topics(H,feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( H[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(documents[doc_index])


In [200]:
x.apply(display_topics(nmf.components_,vectorizer.get_feature_names(),df_art['Title-Cleaned'],100000,1000))

Topic 0:
neue wiener ndigt angek gr sp android mpfe ne greifen regeln gemeindewohnungen apple wieso mass sparsamer effect playstation tb speicher andromeda leichter galaxie aufbruch google fl zwei griechenland ge le vorschl eurosondergipfel chtlinge music botaniker wiesen pflanzenarten smarttvsystem tv kan sen tze aufregung paris bringt doom schaffen kettens brutal genmassaker zwergplanet pl erster ukraine ansicht mysteri cere flecken haus blick kommen softwaregeneration feuer ostukraine teurer fronten tschechows gewehr rfte geschichteprojekt burg not zinsflaute steuerbelastungen anleger versuch reifepr ersten bilanz fung streamingdienst ten sollen raketen fordert stoppen selbstmordpanzerattacken arbeitslose ak strategie reisende entt schlichtungsstelle uschte chat orfstruktur livebericht rden feature kollateralfragen realit unobericht gazakrieg sch lle stra startet ere matura schulen spielen fp wahlkampf antifp nde wowspieler podcast ler lehrer benotet regime dishonor neues killer geb

TypeError: ignored

In [201]:
LD=LatentDirichletAllocation(n_components=4,random_state=42)
fi=LD.fit_transform(x)
fi

array([[0.08339472, 0.74857397, 0.08399237, 0.08403894],
       [0.08339472, 0.74857397, 0.08399237, 0.08403894],
       [0.06450598, 0.80942915, 0.06322398, 0.06284089],
       ...,
       [0.90545788, 0.03149606, 0.03176911, 0.03127694],
       [0.03811158, 0.88814761, 0.03620104, 0.03753977],
       [0.04183747, 0.04183698, 0.04170439, 0.87462116]])

In [0]:
def yolo(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))


In [203]:
x.apply(yolo(LD,vectorizer,9))
#It's working!


Topic #0:
neue gro ber fl sch wien macht kurz sterreich

Topic #1:
ber apple wien verl sterreich eu drei warum gehen

Topic #2:
fp neue sterreich zwei gr europa sp mehr ck

Topic #3:
wiener google griechenland gestorben lt fp mehr erneut erh


TypeError: ignored

new

In [0]:
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

In [0]:
df_art["Clean-Body-NO-html"]  = df_art['Body'].apply(striphtml)

In [206]:
df_art["Clean-Body-NO-html"]

0      Abonnieren Sie einen unserer Newsletter und la...
1      Werden Sie Teil von derStandard.at!10. Dezembe...
2      Die Smartphone-App bietet Nachrichten in Echtz...
3      Wie sieht Ihr Alltag als Linkshänder aus? Erle...
4      Erzählen Sie uns von Ihren Erlebnissen rund um...
                             ...                        
995    Die Lotterien sollen zu 100 Prozent unter ein ...
996    Microsoft-CEO Nadella kündigt "harte Entscheid...
997    Hersteller: "Wenn du mit blutigen Games nichts...
998    Grüner Abgeordneter hatte von flächendeckendem...
999    Zunächst sicherheitshalber noch speziell gesch...
Name: Clean-Body-NO-html, Length: 1000, dtype: object

In [207]:
df_art["Body-Cleaned"]=  df_art["Clean-Body-NO-html"].apply(pipeline.fit_transform)
df_art["Body-Cleaned"]

0      abonnieren unserer newsletter lassen aktuelle ...
1      teil derstandardat dezember function popupwinp...
2      smartphoneapp bietet nachrichten echtzeit m gl...
3      sieht alltag linksh nder erledigen arbeiten re...
4      erz hlen erlebnissen rund weltpolitischen umbr...
                             ...                        
995    lotterien sollen prozent dach konzernchef stos...
996    microsoftceo nadella k ndigt harte entscheidun...
997    hersteller blutigen game anfangen kannst f r d...
998    gr ner abgeordneter fl chendeckendem lauschang...
999    zun chst sicherheitshalber speziell geschulte ...
Name: Body-Cleaned, Length: 1000, dtype: object

In [208]:
df_art.head()

,ID_Article,Path,publishingDate,Title,Body,Title-Cleaned,Clean-Body-NO-html,Body-Cleaned
0,1,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",newsletter derstandardat,Abonnieren Sie einen unserer Newsletter und la...,abonnieren unserer newsletter lassen aktuelle ...
1,2,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ...",teil derstandardat,Werden Sie Teil von derStandard.at!10. Dezembe...,teil derstandardat dezember function popupwinp...
2,3,Diverses/mobil,2013-11-22 12:15:00.00,Die Android App von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",android app derstandardat,Die Smartphone-App bietet Nachrichten in Echtz...,smartphoneapp bietet nachrichten echtzeit m gl...
3,4,Newsroom/User/mitmachen/Mitreden,2014-08-13 05:30:00.00,Welche Erfahrungen haben Sie als Linkshänder g...,"<div class=""section"" id=""content-main"" itempro...",erfahrungen linksh nder gemacht,Wie sieht Ihr Alltag als Linkshänder aus? Erle...,sieht alltag linksh nder erledigen arbeiten re...
4,5,Newsroom/User/mitmachen/Mitreden,2014-08-27 12:27:01.09,Wie haben Sie das Jahr 1989 erlebt?,"<div class=""section"" id=""content-main"" itempro...",jahr erlebt,Erzählen Sie uns von Ihren Erlebnissen rund um...,erz hlen erlebnissen rund weltpolitischen umbr...


In [0]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_art['Body-Cleaned'])
x = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())

In [0]:
model = NMF(n_components=10, init='nndsvd', random_state=0, alpha=0.1, l1_ratio=0.5)
W = model.fit_transform(X)


In [0]:
H = model.components_

In [269]:
W


array([[0.04577673, 0.09874055, 0.        , ..., 0.14617827, 0.04975496,
        0.        ],
       [0.22591633, 0.        , 0.40343905, ..., 0.24833803, 0.        ,
        0.        ],
       [0.02437639, 0.        , 0.10236527, ..., 1.3029696 , 0.        ,
        0.        ],
       ...,
       [0.        , 0.34798149, 0.        , ..., 0.0751363 , 0.10251008,
        0.1008737 ],
       [0.        , 0.07322829, 0.0398994 , ..., 0.        , 0.3151435 ,
        0.00678484],
       [0.12331802, 0.09944214, 0.        , ..., 0.26028635, 0.09718668,
        0.        ]])

In [270]:
H


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00122399, 0.0031307 , ..., 0.00394153, 0.        ,
        0.00910455],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.0011258 , 0.        , 0.        , ..., 0.        , 0.00713047,
        0.0183816 ],
       [0.        , 0.        , 0.        , ..., 0.00023097, 0.        ,
        0.0027899 ]])

In [0]:
dfw=pd.DataFrame(W)

In [272]:
dfw.head()

,0,1,2,3,4,5,6,7,8,9
0,0.045777,0.098741,0.000000,0.036571,0.0,0.056384,0.000000,0.146178,0.049755,0.000000
1,0.225916,0.000000,0.403439,0.000000,0.0,1.329265,0.013268,0.248338,0.000000,0.000000
2,0.024376,0.000000,0.102365,0.000000,0.0,0.091671,0.000000,1.302970,0.000000,0.000000
3,0.006881,0.148619,0.000000,0.000000,0.0,0.065368,0.121021,0.067500,0.102685,0.000000
4,0.004659,0.287533,0.010634,0.006276,0.0,0.060294,0.000000,0.070794,0.049606,0.030252


In [0]:
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(documents[doc_index])

In [276]:
display_topics(H,W,list(x),df_art["Body-Cleaned"],1,10)

Topic 0:
auto
bild tages blasenleiden pazifikansichtssache juni function popupwinpage width height noscroll popup windowopenpage popupwin toolbar nomenubar noscrollbars noscroll resizable yes width width height height leave top popup null popupfocus postingsdie sch nsten skurrilsten r udigsten blo informativen momente mobilit t sammelsurium st ndig erweitert fall foto kriterien entspricht freuen zusendungen bild bild mehr verf gbar blasenleiden pazifik eher mittelerfolgreich verliefen j ngsten test lowdensity supersonic decelerator nasa technik abbremsen gr erer raumtransporter eintritt atmosph re erm glichen dabei zwei bremsvorrichtungen gez ndet erste quasi aufblasbare untertasse stirnfl che flugger t deutlich vergr ert danach kommt klassischer bremsfallschirm einsatz usweltraumbeh rde system landung gr erer transporter mar erm glichen versuchstr ger aktuellen test pazifik hawaii aufschlug einsatztauglich abbremsblase ffnete gew nscht fiel gleich danach zusammenweiter var slideshowpa

In [254]:
!pip install pyldavis


In [279]:
lda_tf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tf.fit(X)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [287]:
import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.sklearn
from __future__ import print_function

pyLDAvis.enable_notebook()

movies_vis_data = pyLDAvis.genism.prepare(lda_tf, X , vectorizer)


AttributeError: ignored